In [1]:
from datascience import *
from prob140 import *

from prob140.pykov import *

# We could also do import prob140.pykov as pk
# and call pk.Vector, etc

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import prob140
prob140.__version__
# Make sure you update

'0.2.0.0'

There's more extensive documentation at https://github.com/riccardoscalco/Pykov

## Construction

In [3]:
"""
If students are more familiar with
v1 = Vector("A", 0.3, "B", 0.7)
we could implement that too

Note that order of contents in Vector is nondeterministic
"""
v1 = Vector(A=0.3, B=0.7)
v1

Vector([('A', 0.3), ('B', 0.7)])

In [4]:
v1['A']

0.3

In [5]:
# Can be initialized with a dictionary
v2 = Vector({'A' : 0.5, 'B' : 0.5})
v2

Vector([('A', 0.5), ('B', 0.5)])

## Vector Operations

In [6]:
v1 + v2

Vector([('A', 0.8), ('B', 1.2)])

In [7]:
v1 - v2

Vector([('A', -0.2), ('B', 0.19999999999999996)])

In [8]:
v1 * 2

Vector([('A', 0.6), ('B', 1.4)])

In [9]:
# Dot product
v1 * v2

0.5

In [10]:
v3 = Vector(C=0.1, D=0.9)
v1 + v3

Vector([('C', 0.1), ('A', 0.3), ('D', 0.9), ('B', 0.7)])

## Vector functions

In [11]:
v1.sum()

1.0

In [12]:
# sorts by probabilities
(v1+v3).sort()

[('C', 0.1), ('A', 0.3), ('B', 0.7), ('D', 0.9)]

In [13]:
# Randomly chooses, weighted by probability
v1.choose()

'B'

In [14]:
[v1.choose() for i in range(10)]

['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'A', 'B']

In [15]:
v4 = v1 + v3
v4.normalize()
v4

Vector([('C', 0.05), ('A', 0.15), ('D', 0.45), ('B', 0.35)])

In [16]:
v4.copy()

Vector([('C', 0.05), ('A', 0.15), ('D', 0.45), ('B', 0.35)])

## Matrices

Initializing is a little bit cumbersome

Maybe we can do something like entering a numpy.matrix or JointDistribution Table style?

In [17]:
T = Matrix({('A','B'): .3, ('A','A'): .7, ('B','A'): 1.})
T

Matrix([(('B', 'A'), 1.0), (('A', 'A'), 0.7), (('A', 'B'), 0.3)])

In [18]:
T * T

Matrix([(('A', 'B'), 0.20999999999999999),
        (('B', 'A'), 0.69999999999999996),
        (('A', 'A'), 0.78999999999999992),
        (('B', 'B'), 0.29999999999999999)])

In [19]:
T + T

Matrix([(('B', 'A'), 2.0), (('A', 'A'), 1.4), (('A', 'B'), 0.6)])

In [20]:
v1 * T

Vector([('A', 0.90999999999999992), ('B', 0.089999999999999997)])

In [21]:
T.states()

{'A', 'B'}

In [22]:
# Predecessors
T.pred()

OrderedDict([('A', Vector([('B', 1.0), ('A', 0.7)])),
             ('B', Vector([('A', 0.3)]))])

In [23]:
# Successors
T.succ()

OrderedDict([('A', Vector([('A', 0.7), ('B', 0.3)])),
             ('B', Vector([('A', 1.0)]))])

In [24]:
# Right stochastic matrix
U = Matrix({('A','B'): 3, ('A','A'): 7, ('B','A'): .2})
U.stochastic()
U

Matrix([(('B', 'A'), 1.0), (('A', 'A'), 0.7), (('A', 'B'), 0.3)])

In [25]:
T.transpose()

Matrix([(('A', 'B'), 1.0), (('A', 'A'), 0.7), (('B', 'A'), 0.3)])

In [26]:
# Identity matrix
T.eye()

Matrix([(('A', 'A'), 1.0), (('B', 'B'), 1.0)])

In [27]:
# Ones vector
T.ones()

Vector([('A', 1.0), ('B', 1.0)])

## Markov Chains

Subclass of Matrix

Needs to be right stochastic

In [28]:
T = Chain({('A','B'): .3, ('A','A'): .7, ('B','A'): 1.})

T.adjacency()

Matrix([(('B', 'A'), 1), (('A', 'A'), 1), (('A', 'B'), 1)])

In [29]:
T.pow(v1, 3)

# Equivalent to v1*T*T*T

Vector([('A', 0.78189999999999993), ('B', 0.21809999999999996)])

In [30]:
# Randomly move between successors
T.move('A')

'B'

In [31]:
# Randomly genera walk of size steps
T.walk(10)

['B', 'A', 'A', 'A', 'A', 'A', 'B', 'A', 'B', 'A', 'A']

In [32]:
T.walk(10, start="A", stop="B")

['A', 'A', 'A', 'A', 'B']

In [33]:
# Steady state
T.steady()

Vector([('A', 0.76923076923076905), ('B', 0.23076923076923092)])